<center><br><h1 style="font-size:50px; font-family:Tahoma;margin-top:50px;position:inherit;">Vision Transformers are Easy</h1></center>
<br>
<center><img src="https://www.iita.org/wp-content/uploads/2019/07/Cassava-Mosaic-Disease.jpg" style="width:70%; border-radius:10px;"></center>

<p style="font-size:22px; font-family:Tahoma;margin-right:50px">Please give some due credit to <a href="https://www.kaggle.com/abhinand05">Abhinand</a> for helping make this Kernel a reality.</p>

In [ ]:
!cp -r ../input/vittutorialillustrations/* ./ 

!pip install nb_black
%load_ext nb_black

<p style="font-size:22px; font-family:Tahoma;margin-right:50px">Installing Pytorch-XLA inorder to access the TPU:</p>

In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version 1.7
!pip install timm

<p style="font-size:22px; font-family:Tahoma;margin-right:50px">Importing some really important stuff in here. I hope you are still awake as you course through this notebook.<p>

In [ ]:
# Numpy, Pandas and MatPlot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("ggplot")

# Who doesn't love PyTorch
import torch
import torch.nn as nn

# For the Transformations
import torchvision.transforms as transforms

# For the TPU to work
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.distributed.parallel_loader as pl

import timm

# Some Native Python Libraries
import gc
import os
import time
import random
from datetime import datetime

from PIL import Image
from tqdm.notebook import tqdm
from sklearn import model_selection, metrics

<p style="font-size:22px; font-family:Tahoma;margin-right:50px">🚨 Jargon Alert 🚨<br><br>I know, TPU Parallelization sounds aweful but you can watch <a href="https://www.youtube.com/watch?v=HgGyWS40g-g">this video</a> about it. Watch till 1:04. It is by folks at TensorFlow so you need not watch it for a very long time.<br>(No one likes TensorFlow or atleast in the world I live in)</p>

In [ ]:
# For parallelization in TPUs
os.environ["XLA_USE_BF16"] = "1"
os.environ["XLA_TENSOR_ALLOCATOR_MAXSIZE"] = "100000000"

<p style="font-size:22px; font-family:Tahoma;margin-right:50px">When we say that: <br>"Hey let's seed this!"<br>
DL Developers aren't really thinking about gardening, or well, uh. It has to do maybe a little bit about the reproducebility of the model.<br><br>Check out this Medium Article:<a href="https://medium.com/@ODSC/properly-setting-the-random-seed-in-ml-experiments-not-as-simple-as-you-might-imagine-219969c84752"><br><i>Properly Setting the Random Seed in ML Experiments. Not as Simple as You Might Imagine</i> 🐵</a><br>to get a better idea.</p>

In [ ]:
def seed_everything(seed):
    """
    Seeds basic parameters for reproductibility of results
    
    Arguments:
        seed {int} -- Number of the seed
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(1001)

<p style="font-size:22px; font-family:Tahoma;margin-right:50px">Specifying necessary paths and other necessities:<p>

In [ ]:
# general global variables
DATA_PATH = "../input/cassava-leaf-disease-classification"
TRAIN_PATH = "../input/cassava-leaf-disease-classification/train_images/"
TEST_PATH = "../input/cassava-leaf-disease-classification/test_images/"
MODEL_PATH = (
    "../input/vit-base-models-pretrained-pytorch/jx_vit_base_p16_224-80ecf9dd.pth"
)

# model specific global variables

# THIS IS THE IMAGE SIZE IN PIXELS:
IMG_SIZE = 224

BATCH_SIZE = 16

LR = 2e-05

N_EPOCHS = 10

In [ ]:
# Lets take a look at the data:
df = pd.read_csv(os.path.join(DATA_PATH, "train.csv"))

# Splitting the dataset into training and testsets:
train_df, valid_df = model_selection.train_test_split(
    df, test_size=0.1, random_state=42, stratify=df.label.values
)

df.head()

<p style="font-size:22px; font-family:Tahoma;margin-right:50px">We will now be visualizing on how
<p style="font-size:22px; font-family:Tahoma;margin-right:50px">Plotting to see how various labels are distributed:</p>

In [ ]:
df.label.value_counts().plot(kind="bar")

<p style="font-size:22px; font-family:Tahoma;margin-right:50px">Defining the CasavaDataset:</p>

In [ ]:
class CassavaDataset(torch.utils.data.Dataset):
    """
    Helper Class to create the pytorch dataset
    """

    def __init__(self, df, data_path=DATA_PATH, mode="train", transforms=None):
        super().__init__()
        self.df_data = df.values
        self.data_path = data_path
        self.transforms = transforms
        self.mode = mode
        self.data_dir = "train_images" if mode == "train" else "test_images"

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, index):
        img_name, label = self.df_data[index]
        img_path = os.path.join(self.data_path, self.data_dir, img_name)
        img = Image.open(img_path).convert("RGB")

        if self.transforms is not None:
            image = self.transforms(img)

        return image, label

<h1 style="font-size:32px; font-family:Tahoma;margin-right:50px">Data Augmentation to go through:</h1>
<p style="font-size:22px; font-family:Tahoma;margin-right:50px">We go through some really particular Data Augmentations. I have to say that our dude <a href="https://www.kaggle.com/abhinand05">Abhinand</a> did an amazing job here. Kudos to him.<br><br>
If you want a better explanation head to the docs of <em style="background-color:#bfbfbf; font-family: 'Courier New', monospace;">torchvision.transforms</em> <a href="https://pytorch.org/docs/stable/torchvision/transforms.html">here</a>.</p>

In [ ]:
# Time to do some data augmentations.

transforms_train = transforms.Compose(
    [
        # Resizing the image to the previously stated size
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        # Randomly flipping the images horizontally with the probability of 30%
        transforms.RandomHorizontalFlip(p=0.3),
        # Randomly flipping the images vertically with the probability of 30%
        transforms.RandomVerticalFlip(p=0.3),
        # Randomly Rotating the images by 10 degrees
        transforms.RandomRotation(10),
        # Randomly 10 degrees worth of Affine
        transforms.RandomAffine(10),
        # Croppping the images to the stated
        transforms.RandomResizedCrop(IMG_SIZE),
        # Converting the image to tensor
        transforms.ToTensor(),
        # Normalizing
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

transforms_valid = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

<h1 style="font-size:32px; font-family:Tahoma;margin-right:50px">Defining the Model</h1>

In [ ]:
class ViTBase16(nn.Module):
    def __init__(self, n_classes, pretrained=False):

        super(ViTBase16, self).__init__()

        self.model = timm.create_model("vit_base_patch16_224", pretrained=False)

        # I mean it is pretrained?!
        if pretrained:
            self.model.load_state_dict(torch.load(MODEL_PATH))

        self.model.head = nn.Linear(self.model.head.in_features, n_classes)

    def forward(self, x):
        x = self.model(x)
        return x

    def train_one_epoch(self, train_loader, criterion, optimizer, device):

        # The Accuracy starts from Zero:
        epoch_loss = 0.0
        epoch_accuracy = 0.0

        # This bad boy trains the model
        self.model.train()
        for i, (data, target) in enumerate(train_loader):

            # Clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = self.forward(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # Calculate Accuracy
            accuracy = (output.argmax(dim=1) == target).float().mean()
            # update training loss and accuracy
            epoch_loss += loss
            epoch_accuracy += accuracy

            # perform a single optimization step (parameter update)
            if device.type == "xla":
                xm.optimizer_step(optimizer)

                if i % 20 == 0:
                    xm.master_print(f"\tBATCH {i+1}/{len(train_loader)} - LOSS: {loss}")

            else:
                optimizer.step()

        return epoch_loss / len(train_loader), epoch_accuracy / len(train_loader)

    def validate_one_epoch(self, valid_loader, criterion, device):
        # keep track of validation loss
        valid_loss = 0.0
        valid_accuracy = 0.0

        ######################
        # validate the model #
        ######################
        self.model.eval()
        for data, target in valid_loader:
            # move tensors to GPU if CUDA is available
            if device.type == "cuda":
                data, target = data.cuda(), target.cuda()
            elif device.type == "xla":
                data = data.to(device, dtype=torch.float32)
                target = target.to(device, dtype=torch.int64)

            with torch.no_grad():
                # forward pass: compute predicted outputs by passing inputs to the model
                output = self.model(data)
                # calculate the batch loss
                loss = criterion(output, target)
                # Calculate Accuracy
                accuracy = (output.argmax(dim=1) == target).float().mean()
                # update average validation loss and accuracy
                valid_loss += loss
                valid_accuracy += accuracy

        return valid_loss / len(valid_loader), valid_accuracy / len(valid_loader)

<h1 style="font-size:32px; font-family:Tahoma;margin-right:50px"> Fitting the Model to the TPU 🏗</h1>

In [ ]:
def fit_tpu(
    model, epochs, device, criterion, optimizer, train_loader, valid_loader=None
):

    valid_loss_min = np.Inf  # track change in validation loss

    # keeping track of losses as it happen
    train_losses = []
    valid_losses = []
    train_accs = []
    valid_accs = []

    for epoch in range(1, epochs + 1):
        gc.collect()
        para_train_loader = pl.ParallelLoader(train_loader, [device])

        xm.master_print(f"{'='*50}")
        xm.master_print(f"EPOCH {epoch} - TRAINING...")
        train_loss, train_acc = model.train_one_epoch(
            para_train_loader.per_device_loader(device), criterion, optimizer, device
        )
        xm.master_print(
            f"\n\t[TRAIN] EPOCH {epoch} - LOSS: {train_loss}, ACCURACY: {train_acc}\n"
        )
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        gc.collect()

        if valid_loader is not None:
            gc.collect()
            para_valid_loader = pl.ParallelLoader(valid_loader, [device])
            xm.master_print(f"EPOCH {epoch} - VALIDATING...")
            valid_loss, valid_acc = model.validate_one_epoch(
                para_valid_loader.per_device_loader(device), criterion, device
            )
            xm.master_print(f"\t[VALID] LOSS: {valid_loss}, ACCURACY: {valid_acc}\n")
            valid_losses.append(valid_loss)
            valid_accs.append(valid_acc)
            gc.collect()

            # save model if validation loss has decreased
            if valid_loss <= valid_loss_min and epoch != 1:
                xm.master_print(
                    "Validation loss decreased ({:.4f} --> {:.4f}).  Saving model ...".format(
                        valid_loss_min, valid_loss
                    )
                )
            #                 xm.save(model.state_dict(), 'best_model.pth')

            valid_loss_min = valid_loss

    return {
        "train_loss": train_losses,
        "valid_losses": valid_losses,
        "train_acc": train_accs,
        "valid_acc": valid_accs,
    }

<p style="font-size:22px; font-family:Tahoma;margin-right:50px">Grabbing the Model</p>

In [ ]:
model = ViTBase16(n_classes=5, pretrained=True)

<h1 style="font-size:32px; font-family:Tahoma;margin-right:50px">Training the Model</h1>

In [ ]:
def _run():
    train_dataset = CassavaDataset(train_df, transforms=transforms_train)
    valid_dataset = CassavaDataset(valid_df, transforms=transforms_valid)

    train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=True,
    )

    valid_sampler = torch.utils.data.distributed.DistributedSampler(
        valid_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=False,
    )

    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        sampler=train_sampler,
        drop_last=True,
        num_workers=8,
    )

    valid_loader = torch.utils.data.DataLoader(
        dataset=valid_dataset,
        batch_size=BATCH_SIZE,
        sampler=valid_sampler,
        drop_last=True,
        num_workers=8,
    )

    criterion = nn.CrossEntropyLoss()
    #     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = xm.xla_device()
    model.to(device)

    lr = LR * xm.xrt_world_size()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    xm.master_print(f"INITIALIZING TRAINING ON {xm.xrt_world_size()} TPU CORES")
    start_time = datetime.now()
    xm.master_print(f"Start Time: {start_time}")

    logs = fit_tpu(
        model=model,
        epochs=N_EPOCHS,
        device=device,
        criterion=criterion,
        optimizer=optimizer,
        train_loader=train_loader,
        valid_loader=valid_loader,
    )

    xm.master_print(f"Execution time: {datetime.now() - start_time}")

    xm.master_print("Saving Model")
    xm.save(
        model.state_dict(), f'model_5e_{datetime.now().strftime("%Y%m%d-%H%M")}.pth'
    )

In [ ]:
# Start training processes
def _mp_fn(rank, flags):
    torch.set_default_tensor_type("torch.FloatTensor")
    a = _run()


# _run()
FLAGS = {}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method="fork")

<p style="font-size:22px; font-family:Tahoma;margin-right:50px">Well folks, that would be it, I'll update this every once in a while to see if there's more scope for improvement. I will not be using this notebook to make submissions because parts of it are code that were adapted from other's works. Please give some due credit to <a href="https://www.kaggle.com/abhinand05">Abhinand</a> too. <br><br>Thanks for going through the notebook. An upvote would be appreciated 😋</p>